In [ ]:
#! /usr/bin/env python3

In [ ]:
# Author: Connor Riley
# Email: connor.riley2@gmail.com
#
# Richmond Real Estate Tax Assessments Analysis
# Version 1.0

In [1]:
import numpy as np
import pandas as pd
#import shapefile
import timeit

import helper

In [2]:
pd.set_option('display.max_columns', None)

In [ ]:
# years = [2017, 2016, 2015]#df.year.unique().tolist():
# setBuckets(df_agg, years,'med_TOTAL_VALUE', 'med_VALUE_Buckets')

In [3]:
path_ass_hist = "./assets/Property_Assessment_History.csv"
path_ass_cur = "./assets/Property_Assessments_Current.csv"
path_tran_hist = "./assets/Property_Transfer_History.csv"
path_geo = "./assets/Parcel_Geographic_Summary.csv"
# path3 = "./assets/Neighborhoods.csv"

In [4]:
df_ass_hist = pd.read_csv(path_ass_hist, header=0, low_memory=False)
df_ass_hist.shape

(1418107, 30)

In [5]:
#Removing duplicate 
#before (1418107, 30)
df_ass_hist = df_ass_hist.drop_duplicates()
df_ass_hist.shape

(1411989, 30)

In [6]:
df_ass_cur = pd.read_csv(path_ass_cur, header=0, low_memory=False)
df_ass_cur.shape

(76393, 35)

In [7]:
#Removing duplicate 
#before (76393, 30)
df_ass_cur = df_ass_cur.drop_duplicates()
df_ass_cur.shape

(74407, 35)

In [8]:
#get common columns for union
frames = [df_ass_hist, df_ass_cur]
common_cols = list(set.intersection(*(set(df.columns) for df in frames)))

In [9]:
df_ass = pd.concat([df[common_cols] for df in frames], ignore_index=True).drop_duplicates()

In [10]:
df_ass.shape

(1381970, 24)

In [11]:
df_tran_hist = pd.read_csv(path_tran_hist, header=0, low_memory=False)
df_tran_hist.shape

(266766, 6)

In [12]:
#Removing duplicate 
#before (266766, 30)
df_tran_hist = df_tran_hist.drop_duplicates()
df_tran_hist.shape

(266603, 6)

In [19]:
df_ass.head(2)

,SPECIAL_ASSESSMENT_DISTRICT,ASSESSMENT_MAILING_ZIP_CODE,PROPERTY_CLASSCODE,AREA_TAX,TOTAL_VALUE,LAND_SQFT,ASSESSMENT_NEIGHBORHOOD_CODE,PIN,ASSESSMENT_MAILING_CITY,STATE_PLANE_X_COORDINATE,ASSESSMENT_MAILING_ADDRESS,SPECIAL_ASSESSMENT_DIST_DESC,Location 1,OWNER_NAME,CORPORATION_ID,ASSESSORS_NEIGHBORHOOD_DESC,STATE_PLANE_Y_COORDINATE,PROPERTY_CLASS_DESCRIPTION,ASSESSMENT_DATE,LAND_VALUE,JURISDICTION_ID,GPIN,DWELLING_VALUE,ASSESSMENT_MAILING_STATE
0,NaN,2322700000,512,NaN,$222700.00,NaN,NaN,N0170235011,RICHMOND,1.178472e+07,4313 FAUQUIER AVE,NaN,"(37.59309107, -77.4585498)",RAPPAPORT GREGORY J & ELIZABETH J,NaN,NaN,3.741178e+06,"512 Single Family, 2 Story",01/01/2005,$53800.00,NaN,21045.0,$168900.00,VA
1,NaN,2322500000,468,NaN,$479000.00,NaN,NaN,C0040945026,RICHMOND,1.176883e+07,P O BOX 9014,NaN,"(37.53282771, -77.51368659)",GRANITE RECREATION CORP,NaN,NaN,3.719112e+06,B468 Com Community Ctr-Private,01/01/2006,$92700.00,NaN,4031.0,$386300.00,VA


In [14]:
df_tran_hist.head(2)

,PIN,SALE_DATE,SALE_AMOUNT,GRANTOR,SALE_REFERENCE_DOCUMENT,VALIDATED_TRANS_DESC
0,S0002486009,11/01/1991,NaN,NaN,000022-00658,NaN
1,W0000016013B,04/01/2004,$1356000.00,GENERAL ELECTRIC CAPITAL,ID2004-10245,NaN


In [15]:
df_tran_hist['year'] = df_tran_hist['SALE_DATE'].str.slice(6,10)

In [16]:
df_geo = pd.read_csv(path_geo, header=0, low_memory=False)
df_geo.shape

(74262, 69)

In [17]:
df_geo.head(2)

,PIN_PIN,ARTINCZ_Name,BG10_ID,BG00_ID,BG90_ID,BLOCK10_Name,BLOCK00_Name,BLOCK90_Name,BULK_Schedule,CARE_Name,CHESBAY_Flag,CIVIC_ID1,CIVIC_ID2,CIVIC_ID3,CIVIC_ID4,CIVIC_ID5,CIVIC_Name1,CIVIC_Name2,CIVIC_Name3,CIVIC_Name4,CIVIC_Name5,CODEINSP_ID,CONGRESS_Name,COUNCIL_Name,CUP_Flag,DESOVER_Name,DISPTCHZ_Name,EZ_Name,FIREDIST_Name,FLOOD500_Flag,HDCITY_Name,HDNATREG_Name,HOA_Name,HOUSE00_Name,LEAF_Schedule,LUPLAN_ID,LUPLAN_Name,NGHOOD_ID,NGHOOD_Name,NIBIA_Name,NIBTA_Name,PLGDIST_Name,POLPREC_Name,POLSECTR_Name,POLTAZ_Name,RCA_Name,REFUSE_ID,SCHZELEM_ID,SCHZELEM_Name,SCHZHIGH_ID,SCHZHIGH_Name,SCHZMID_ID,SCHZMID_Name,SENATE00_Name,SPASMTDT_Name,SUP_Flag,SWEEP_Schedule,TAZ_Name,TRACT10_ID,TRACT10_Name,TRACT00_ID,TRACT00_Name,TRACT90_ID,TRACT90_Name,VOTPREC_Name,WETLAND_Flag,ZIP_Name,ZONING_Name,Location 1
0,W0000736066,NaN,404001,404001,404001.0,1001,1002,103,NaN,NaN,N,350.0,770.0,NaN,NaN,NaN,Fan District Association,Monument Avenue Preservation Society,NaN,NaN,NaN,3026,4,2,NaN,NaN,034A,NaN,10,N,Monument Avenue,Fan Area,NaN,71,TBD,SF-MD,Single-Family (Medium Density),FAN,The Fan,NaN,NaN,Near West,3,313,34,NaN,Wednesday,317,Fox,102,Jefferson,202,Binford,9,NaN,NaN,TBD,1073,40400,404.0,40400,404.0,40400.0,404.0,204,N,23220,R-48,"(37.554815, -77.458805)"
1,N0000557012,NaN,109003,109003,109003.0,3015,3012,320,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4028,4,6,NaN,NaN,093B,NaN,15,N,NaN,Chestnut Hill-Plateau,NaN,71,TBD,SF-LD,Single-Family (Low Density),HPST,Highland Park Southern Tip,Highland Park Southern Tip,Highland Park Southern Tip,North,4,411,93,Highland Park,Wednesday,335,Overby Sheppard,108,Armstrong,212,Martin Luther King Jr,9,NaN,NaN,TBD,1025,10900,109.0,10900,109.0,10900.0,109.0,604,N,23222,R-5,"(37.563372, -77.424223)"


In [20]:
df_geo = df_geo.drop(['ARTINCZ_Name', 'Location 1','SWEEP_Schedule','SCHZMID_Name','SCHZHIGH_Name','SCHZELEM_Name','REFUSE_ID','BG00_ID','BG90_ID', 'BLOCK00_Name','BLOCK90_Name','BULK_Schedule','CIVIC_ID1','CIVIC_ID2','CIVIC_ID3','CIVIC_ID4','CIVIC_ID5','CIVIC_Name1','CIVIC_Name2','CIVIC_Name3','CIVIC_Name4','CIVIC_Name5','CODEINSP_ID','CONGRESS_Name','COUNCIL_Name','DISPTCHZ_Name','FIREDIST_Name','FLOOD500_Flag','HOA_Name','HOUSE00_Name','LEAF_Schedule','LUPLAN_ID','NGHOOD_ID','NIBIA_Name','NIBTA_Name','PLGDIST_Name','POLPREC_Name','POLSECTR_Name','POLTAZ_Name','SENATE00_Name','TAZ_Name','TRACT10_Name','TRACT00_ID','TRACT00_Name','TRACT90_ID','TRACT90_Name','VOTPREC_Name','WETLAND_Flag'], axis=1)

In [21]:
#Removing duplicate 
#before (1341903, 31)
df_geo = df_geo.drop_duplicates()
df_geo.shape

(74262, 21)

In [22]:
df_mix = pd.merge(df_geo,df_ass, left_on=['PIN_PIN'], right_on=['PIN'], how="left").drop_duplicates()

In [23]:
df_mix.head(2)

,PIN_PIN,BG10_ID,BLOCK10_Name,CARE_Name,CHESBAY_Flag,CUP_Flag,DESOVER_Name,EZ_Name,HDCITY_Name,HDNATREG_Name,LUPLAN_Name,NGHOOD_Name,RCA_Name,SCHZELEM_ID,SCHZHIGH_ID,SCHZMID_ID,SPASMTDT_Name,SUP_Flag,TRACT10_ID,ZIP_Name,ZONING_Name,SPECIAL_ASSESSMENT_DISTRICT,ASSESSMENT_MAILING_ZIP_CODE,PROPERTY_CLASSCODE,AREA_TAX,TOTAL_VALUE,LAND_SQFT,ASSESSMENT_NEIGHBORHOOD_CODE,PIN,ASSESSMENT_MAILING_CITY,STATE_PLANE_X_COORDINATE,ASSESSMENT_MAILING_ADDRESS,SPECIAL_ASSESSMENT_DIST_DESC,Location 1,OWNER_NAME,CORPORATION_ID,ASSESSORS_NEIGHBORHOOD_DESC,STATE_PLANE_Y_COORDINATE,PROPERTY_CLASS_DESCRIPTION,ASSESSMENT_DATE,LAND_VALUE,JURISDICTION_ID,GPIN,DWELLING_VALUE,ASSESSMENT_MAILING_STATE
0,W0000736066,404001,1001,NaN,N,NaN,NaN,NaN,Monument Avenue,Fan Area,Single-Family (Medium Density),The Fan,NaN,317,102,202,NaN,NaN,40400,23220,R-48,NaN,23832,212.0,NaN,$645000.00,NaN,NaN,W0000736066,CHESTERFIELD,1.178474e+07,PO BOX 255,NaN,"(37.55472362, -77.45891347)",JLM PROPERTIES LLC,NaN,NaN,3.727230e+06,212 Apartment 5-11 Converted,01/01/2005,$90000.00,NaN,58411.0,$555000.00,VA
1,W0000736066,404001,1001,NaN,N,NaN,NaN,NaN,Monument Avenue,Fan Area,Single-Family (Medium Density),The Fan,NaN,317,102,202,NaN,NaN,40400,23220,R-48,0.0,23220,205.0,1.58,$2000.00,3920.0,216.0,W0000736066,RICHMOND,1.178473e+07,1652 W GRACE ST,None,"(37.55472362, -77.45891347)",BRUNSWICK CONDOMINIUM UNIT OWNERS ASSOCIATION,699.0,Fan District,3.727230e+06,R Condo Common Area,01/01/2009,$1000.00,0.0,95586.0,$1000.00,VA


In [24]:
df_mix['year'] = df_mix['ASSESSMENT_DATE'].str.slice(6,10)

In [25]:
df_mix.shape

(1379292, 46)

In [26]:
df = pd.merge(df_mix,df_tran_hist, left_on=["PIN_PIN","year"], right_on=["PIN","year"], how="left")

In [27]:
col_2_drop = ['PIN_x','STATE_PLANE_Y_COORDINATE','STATE_PLANE_X_COORDINATE','PIN_y','SALE_REFERENCE_DOCUMENT','GRANTOR','OWNER_NAME','ASSESSMENT_MAILING_ADDRESS','ASSESSMENT_MAILING_CITY','ASSESSMENT_MAILING_STATE','GRANTOR','OWNER_NAME']
df = df.drop(col_2_drop,axis=1)

In [ ]:
df['LAND_VALUE'] = df['LAND_VALUE'].str.slice(1)
df['DWELLING_VALUE'] = df['DWELLING_VALUE'].str.slice(1)
df['TOTAL_VALUE'] = df['TOTAL_VALUE'].str.slice(1)

In [28]:
df.dtypes

PIN_PIN                          object
BG10_ID                           int64
BLOCK10_Name                      int64
CARE_Name                        object
CHESBAY_Flag                     object
CUP_Flag                        float64
DESOVER_Name                     object
EZ_Name                          object
HDCITY_Name                      object
HDNATREG_Name                    object
LUPLAN_Name                      object
NGHOOD_Name                      object
RCA_Name                         object
SCHZELEM_ID                       int64
SCHZHIGH_ID                       int64
SCHZMID_ID                        int64
SPASMTDT_Name                    object
SUP_Flag                        float64
TRACT10_ID                        int64
ZIP_Name                          int64
ZONING_Name                      object
SPECIAL_ASSESSMENT_DISTRICT     float64
ASSESSMENT_MAILING_ZIP_CODE      object
PROPERTY_CLASSCODE              float64
AREA_TAX                         object


In [ ]:
df = changeToNumeric(df, ['year', 'LAND_VALUE', 'DWELLING_VALUE', 'TOTAL_VALUE', 'STORIES'])

In [ ]:
savePickle(df, "prop_assess_history.p")

In [ ]:
df[:2]

In [ ]:
# Creating Aggregated table
dftmp = df[['NGHOOD_Name', 'year','LAND_VALUE','DWELLING_VALUE','TOTAL_VALUE','LAND_SQFT',
            'BUILDINGS_ON_PROPERTY_COUNT','STORIES', 'YEAR_BUILT', 'SQFT_OF_FINISHED_LIVING_AREA',
            'PROPERTY_CLASS_DESCRIPTION']]
df_agg = dftmp.groupby(['NGHOOD_Name', 'year']).agg('median').add_prefix('med_')\
.join(dftmp.groupby(['NGHOOD_Name', 'year']).median().pct_change().add_prefix('med_pct_chng_'))\
.join(dftmp.groupby(['NGHOOD_Name', 'year']).agg('sum').add_prefix('sum_'))\
.join(dftmp.groupby(['NGHOOD_Name', 'year']).sum().pct_change().add_prefix('sum_pct_chng_'))\
.join(dftmp[['NGHOOD_Name', 'year', 'TOTAL_VALUE']].groupby(['NGHOOD_Name', 'year'])\
     .count().rename(columns={"TOTAL_VALUE": "Property_Count"}))\
.join(dftmp[['NGHOOD_Name', 'year', 'TOTAL_VALUE']].groupby(['NGHOOD_Name', 'year'])\
     .count().pct_change().rename(columns={"TOTAL_VALUE": "pct_chng_Property_Count"}))\
.reset_index()\
.merge( modes(dftmp, ['NGHOOD_Name', 'year'], 'PROPERTY_CLASS_DESCRIPTION', 'mf_prop_class_count')\
     , how='left')

df_agg = df_agg.drop([
    'sum_YEAR_BUILT',
    'sum_STORIES',
    'sum_BUILDINGS_ON_PROPERTY_COUNT'
], axis=1)

In [ ]:
df_agg[['year', 'sum_DWELLING_VALUE', 'med_DWELLING_VALUE', 'sum_pct_chng_DWELLING_VALUE', 'med_pct_chng_DWELLING_VALUE']][:5]

In [ ]:
df_agg[:2]

In [ ]:
dat = shapefile.Reader('./assets/geo_export_2cca3db2-115c-4f41-9f5f-313608e05e80.shp')

In [ ]:
# getParts() and getDict() from http://www.abisen.com/blog/bokeh-maps/

# Given a shapeObject return a list of list for latitude and longitudes values
#       - Handle scenarios where there are multiple parts to a shapeObj


def getParts ( shapeObj ):

    points = []

    num_parts = len( shapeObj.parts )
    end = len( shapeObj.points ) - 1
    segments = list( shapeObj.parts ) + [ end ]

    for i in range( num_parts ):
        points.append( shapeObj.points[ segments[i]:segments[i+1] ] )


    return points


def getDict ( state_name, shapefile ):

    stateDict = {state_name: {} }

    rec = []
    shp = []
    points = []


    # Select only the records representing the
    # "state_name" and discard all other
    for i in shapefile.shapeRecords( ):

        if i.record[4] == state_name:
            rec.append(i.record)
            shp.append(i.shape)

    # In a multi record state for calculating total area
    # sum up the area of all the individual records
    #        - first record element represents area in cms^2
    total_area = sum( [float(i[0]) for i in rec] ) / (1000*1000)


    # For each selected shape object get
    # list of points while considering the cases where there may be
    # multiple parts  in a single record
    for j in shp:
        for i in getParts(j):
            points.append(i)

    # Prepare the dictionary
    # Seperate the points into two separate lists of lists (easier for bokeh to consume)
    #      - one representing latitudes
    #      - second representing longitudes

    lat = []
    lng = []
    for k in points:
        lat.append( [l[1] for l in k] )
        lng.append( [l[0] for l in k] )


    stateDict[state_name]['lat_list'] = lat
    stateDict[state_name]['lng_list'] = lng
    stateDict[state_name]['total_area'] = total_area

    return stateDict


In [ ]:
df_agg['lat'] = 'obj'
df_agg['lng'] = 'obj'

hoods = df_agg.NGHOOD_Name.unique().tolist()

for hood in hoods:
    data = getDict(hood, dat)

    indexes = df_agg.loc[(df_agg.NGHOOD_Name==hood)].index.tolist()
    for ind in indexes:
        df_agg.set_value(ind, 'lat', data[hood]['lat_list'][0])
        df_agg.set_value(ind, 'lng', data[hood]['lng_list'][0])

In [ ]:
savePickle(df_agg, "AGG_prop_hist.p")

In [ ]:
# #function to get .2, .4, .6, and .8 quantiles
# def getQuants(df, col):
#     return df[col].dropna().quantile([.2,.4,.6,.8]).tolist()

In [ ]:
# def setBuckets(df, col, new_col):
#     quants = getQuants(df, col)
#     qRows = []
#     print("Calculated bins for "+col+": ", quants)
    
#     qRows.append( df.loc[ (df[col]>=0) & (df[col]<=quants[0])  ].index.values.tolist() )
#     qRows.append(df.loc[ (df[col]>quants[0]) & (df[col]<=quants[1]) ].index.values.tolist()  )
#     qRows.append(df.loc[ (df[col]>quants[1]) & (df[col]<=quants[2]) ].index.values.tolist()  )
#     qRows.append(df.loc[ (df[col]>quants[2]) & (df[col]<=quants[3]) ].index.values.tolist()  )
#     qRows.append(df.loc[ (df[col]>quants[3]) ].index.values.tolist()  )

#         print("Entries per respective bin: ",len(qRows[0])," - ",len(qRows[1])," - ",len(qRows[2])," - ",len(qRows[3])," - ",len(qRows[4]))
    
            
# #     return df
#     # Values must set in this manner to prevent Pandas from A trying to set a value on a copy of 
#     # a slice from a DataFrame. 
#     for i,qRow in enumerate(qRows):
#         tic=timeit.default_timer()
#         #print("i++ ", i)
#         bucket = 0
#         if i == 0:
#             bucket = 1
#         elif i == 1:
#             bucket = 2
#         elif i == 2:
#             bucket = 3
#         elif i == 3:
#             bucket = 4
#         elif i == 4:
#             bucket = 5
#         else:
#             bucket = 0
#         print("Setting bucket ", bucket, " of 5.")    
#         for ind in qRow:
# #             if j % 10000 ==0 :
# #                 print(j)
# #                 toc=timeit.default_timer()
# #                 print("min: ",(toc - tic) / 60)
# #             df.set_value(ind, new_col, bucket)
#             df.loc[ind, new_col] = bucket
#         btoc=timeit.default_timer()
#         print("Bucket set time in min: ",(btoc - tic) / 60)
#     ttoc=timeit.default_timer()
#     print("Total time in min: ",(ttoc - tic) / 60)
    
            